In [11]:
#Import Libraries
import cv2
import numpy as np
from PIL import Image
from PIL import ImageCms
from PIL import ImageFile
def overlay_image_alpha(img, img_overlay, x, y, alpha_mask):
#Overlay `img_overlay` onto `img` at (x, y) and blend using `alpha_mask`.
#`alpha_mask` must have same HxW as `img_overlay` and values in range [0, 1].
    # Image ranges
    y1, y2 = max(0, y), min(img.shape[0], y + img_overlay.shape[0])
    x1, x2 = max(0, x), min(img.shape[1], x + img_overlay.shape[1])

    # Overlay ranges
    y1o, y2o = max(0, -y), min(img_overlay.shape[0], img.shape[0] - y)
    x1o, x2o = max(0, -x), min(img_overlay.shape[1], img.shape[1] - x)

    # Exit if nothing to do
    if y1 >= y2 or x1 >= x2 or y1o >= y2o or x1o >= x2o:
        return

    # Blend overlay within the determined ranges
    img_crop = img[y1:y2, x1:x2]
    img_overlay_crop = img_overlay[y1o:y2o, x1o:x2o]
    alpha = alpha_mask[y1o:y2o, x1o:x2o, np.newaxis]
    alpha_inv = 1.0 - alpha

    img_crop[:] = alpha * img_overlay_crop + alpha_inv * img_crop
    
# Import Theft Object  
# force opening truncated/corrupt image files
ImageFile.LOAD_TRUNCATED_IMAGES = True
img = cv2.imread("/Users/swathisomayaji/Desktop/BaggageAI_CV_Hiring_Assignment/threat_images/BAGGAGE_20170524_075554_80428_B.jpg", 1)

# PIL image
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

## (1) Convert to gray, and threshold
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
th, threshed = cv2.threshold(gray, 255, 240, cv2.THRESH_BINARY_INV)

## (2) Morph-op to remove noise
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
morphed = cv2.morphologyEx(threshed, cv2.MORPH_CLOSE, kernel)

## (3) Find the max-area contour
cnts = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
cnt = sorted(cnts, key=cv2.contourArea)[-1]

## (4) Crop
x,y,w,h = cv2.boundingRect(cnt)
dst = img[y:y+h, x:x+w]
# add border/padding around the cropped image
dst = cv2.copyMakeBorder(dst, 10, 10, 10, 10, cv2.BORDER_CONSTANT, value=[255,255,255])
#Creating Object with array interface
img = Image.fromarray(dst, 'RGB')
#To make the image background transparent, we first need to change "RGB" to "RGBA".  
img = img.convert("RGBA")
#Rotate the Image by 45 degree
#applying the positive of the angle to anti-rotate clockwise
img=img.rotate(45, resample=0, expand=1, center=None, translate=None, fillcolor=None)
#Then we first take out the pixel value of this picture and determine whether the pixel is white (255, 255, 255)
datas = img.getdata()
# Transparency 
newData = []
  
for items in datas:
    if items[0] == 255 and items[1] == 255 and items[2] == 255:
        newData.append((0, 0, 0, 0))
    else:
        if items[0] >150 and items[1] > 150 and items[2] > 225:
            newData.append((0, 0, 0, 0))
        else:
            newData.append(items)
            #print(items)
img.putdata(newData)
#I replaced all dots with pixel values (255, 255, 255) with transparent
#Import Background Image
background = Image.open("/Users/swathisomayaji/Desktop/BaggageAI_CV_Hiring_Assignment/background_images/S0300542812_20180822020845_L-10_1.jpg").convert("RGBA")
#Find the size of Background image so that size of theft object can be decided. 
bg_w, bg_h = background.size

basewidth = background.size[0]
wpercent = (basewidth/float(img.size[0]))
hsize = int((float(img.size[1])*float(wpercent)))
#Resize of the theft object and assinging it as foreground image.
foreground = img.resize((basewidth,hsize), Image.ANTIALIAS)
#Find the size of foreground image to 
img_w, img_h = foreground.size
x,y = ( (bg_h - img_h) // 2,(bg_w - img_w) // 2)
img = np.array(background)
img_overlay_rgba = np.array(foreground)
#img should not contain an alpha channel. (e.g. If it is RGBA, convert to RGB first.)
#img_overlay has the same number of channels as img.
# Perform blending
alpha_mask = img_overlay_rgba[:, :, 3] / 255.0
img_result = img[:, :, :3].copy()
img_overlay = img_overlay_rgba[:, :, :3]
overlay_image_alpha(img_result, img_overlay, x, y, alpha_mask*0.7)

# Save result
Image.fromarray(img_result).save("img_result.jpg")